# 🗑️ ПОЛНАЯ VLM для мусора

**Ваш вклад = ВСЯ CV часть:**
1. Ансамбль детекторов (YOLO + RT-DETR) → мусор
2. Классификатор сцен (MobileNet) → terrain

**Результат:** "There is 2 plastic on the grass"


In [ ]:
# 1. Установка
!pip install -q roboflow ultralytics supervision transformers pillow tqdm
!git lfs install
!git clone https://github.com/sry4theweight/VLM_garbage.git
%cd VLM_garbage

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# 2. Скачать датасет мусора
!python download_roboflow_dataset.py


In [ ]:
# 3. Скачать датасет сцен (Roboflow Terrain Classification)
!python download_scene_dataset.py --download


In [ ]:
# 4. Обучение классификатора сцен (~5-10 мин на GPU)
!python train_scene_classifier.py --train_dir data/scene_dataset/train --val_dir data/scene_dataset/val --output models/scene_classifier.pt --epochs 15 --batch_size 32


In [ ]:
# 5. Загрузка полной VLM
from VLM_Complete import CompleteVLM
from pathlib import Path
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

vlm = CompleteVLM(
    yolo_path="models/yolo/yolov8x/best.pt",
    detr_path="models/rt-detr/rt-detr-101/m",
    detr_processor_path="models/rt-detr/rt-detr-101/p",
    scene_classifier_path="models/scene_classifier.pt",
    conf_threshold=0.5
)


In [ ]:
# 6. Собираем изображения БЕЗ аугментаций
valid = Path("data/roboflow_dataset/valid")
test = Path("data/roboflow_dataset/test")
images = list(valid.glob("*.jpg")) + list(test.glob("*.jpg"))
print(f"✅ {len(images)} изображений без аугментаций")

COLORS = {'glass':'green', 'plastic':'red', 'metal':'cyan', 'paper':'yellow', 'organic':'lime'}

def test_image(img_path):
    img = Image.open(img_path).convert('RGB')
    analysis = vlm.get_full_analysis(img)
    
    fig, ax = plt.subplots(figsize=(10,7))
    ax.imshow(img)
    for d in analysis['garbage']['detections']:
        b = d['box']
        c = COLORS.get(d['label'], 'white')
        ax.add_patch(patches.Rectangle((b[0],b[1]), b[2]-b[0], b[3]-b[1], lw=2, ec=c, fc='none'))
        ax.text(b[0], b[1]-5, f"{d['label']} {d['confidence']:.0%}", color=c, fontsize=9, weight='bold')
    
    # Показываем сцену только если уверенность >= 80%
    scene_info = analysis['scene']
    if scene_info['confidence'] >= 0.8:
        title = f"Scene: {scene_info['class']} ({scene_info['confidence']:.0%}) | Objects: {analysis['garbage']['total']}"
    else:
        title = f"Scene: uncertain | Objects: {analysis['garbage']['total']}"
    ax.set_title(title)
    ax.axis('off')
    plt.show()
    
    print(f"\n📝 {analysis['description']}")


In [ ]:
# 7. Тест на 3 случайных изображениях
for img in random.sample(images, min(3, len(images))):
    print(f"\n{'='*60}\n📷 {img.name}")
    test_image(img)


In [ ]:
# 8. Интерактивный тест с вопросами
img = random.choice(images)
test_image(img)

questions = ["What is in this image?", "Is there any plastic?", "How many objects?", "Where is the garbage?"]
print("\n💬 Вопросы и ответы:")
for q in questions:
    print(f"\n❓ {q}")
    print(f"✅ {vlm.answer(str(img), q)}")


In [ ]:
# 9. Сохранение на Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/VLM_garbage/models
!cp models/scene_classifier.pt /content/drive/MyDrive/VLM_garbage/models/
!cp -r models/yolo models/rt-detr /content/drive/MyDrive/VLM_garbage/models/
!cp VLM_Complete.py train_scene_classifier.py /content/drive/MyDrive/VLM_garbage/
print("✅ Сохранено на Google Drive!")


# 🗑️ ПОЛНАЯ VLM для мусора

**Ваш вклад = ВСЯ CV часть:**
1. Ансамбль детекторов (YOLO + RT-DETR) → мусор
2. Классификатор сцен (MobileNet) → grass, road, sand...

**Результат:** "There is 2 plastic on the grass"


In [ ]:
# 1. Установка
!pip install -q kagglehub ultralytics supervision transformers pillow roboflow tqdm
!git lfs install
!git clone https://github.com/sry4theweight/VLM_garbage.git
%cd VLM_garbage

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# 2. Скачать датасет мусора
!python download_roboflow_dataset.py


In [ ]:
# 3. Скачать датасет сцен (Intel Image Classification)
!python download_scene_dataset.py --download


In [ ]:
# 4. Обучение классификатора сцен (~5-10 минут на GPU)
!python train_scene_classifier.py --train_dir data/scene_dataset/train --val_dir data/scene_dataset/val --output models/scene_classifier.pt --epochs 10 --batch_size 32


In [ ]:
# 5. Загрузка полной VLM
from VLM_Complete import CompleteVLM
from pathlib import Path
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

vlm = CompleteVLM(
    yolo_path="models/yolo/yolov8x/best.pt",
    detr_path="models/rt-detr/rt-detr-101/m",
    detr_processor_path="models/rt-detr/rt-detr-101/p",
    scene_classifier_path="models/scene_classifier.pt",
    conf_threshold=0.5
)


In [ ]:
# 6. Тестирование на изображениях БЕЗ аугментаций
valid = Path("data/roboflow_dataset/valid")
test = Path("data/roboflow_dataset/test")
images = list(valid.glob("*.jpg")) + list(test.glob("*.jpg"))
print(f"✅ {len(images)} изображений без аугментаций")

COLORS = {'glass':'green', 'plastic':'red', 'metal':'cyan', 'paper':'yellow', 'organic':'lime'}

def test_image(img_path):
    img = Image.open(img_path).convert('RGB')
    analysis = vlm.get_full_analysis(img)
    
    fig, ax = plt.subplots(figsize=(10,7))
    ax.imshow(img)
    for d in analysis['garbage']['detections']:
        b = d['box']
        c = COLORS.get(d['label'], 'white')
        ax.add_patch(patches.Rectangle((b[0],b[1]), b[2]-b[0], b[3]-b[1], lw=2, ec=c, fc='none'))
        ax.text(b[0], b[1]-5, f"{d['label']} {d['confidence']:.0%}", color=c, fontsize=9, weight='bold')
    ax.set_title(f"Scene: {analysis['scene']['class']} | Objects: {analysis['garbage']['total']}")
    ax.axis('off')
    plt.show()
    
    print(f"\n📝 {analysis['description']}")


In [ ]:
# 7. Тест на 3 случайных изображениях
for img in random.sample(images, min(3, len(images))):
    print(f"\n{'='*60}\n📷 {img.name}")
    test_image(img)


In [ ]:
# 8. Интерактивные вопросы
img = random.choice(images)
test_image(img)

questions = [
    "What is in this image?",
    "Is there any plastic?",
    "How many objects?",
    "Where is the garbage?",  # Использует классификатор сцен!
]

print("\n💬 Вопросы и ответы:")
for q in questions:
    print(f"\n❓ {q}")
    print(f"✅ {vlm.answer(str(img), q)}")


In [ ]:
# 9. Сохранение на Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/VLM_garbage/models
!cp models/scene_classifier.pt /content/drive/MyDrive/VLM_garbage/models/
!cp -r models/yolo models/rt-detr /content/drive/MyDrive/VLM_garbage/models/
!cp VLM_Complete.py VLM_Honest.py /content/drive/MyDrive/VLM_garbage/
print("✅ Сохранено на Google Drive!")
